# Linear Regression

Turing is powerful when applied to complex hierarchical models, but it can also be put to task at common statistical procedures, like [linear regression](https://en.wikipedia.org/wiki/Linear_regression). This tutorial covers how to implement a linear regression model in Turing.

We begin by importing all the necessary libraries.

In [2]:
# Import Turing and Distributions.
using Turing, Distributions

# Import RDatasets.
using RDatasets

# Import MCMCChain, Plots, and StatPlots for visualizations and diagnostics.
using MCMCChain, Plots, StatPlots

# MLDataUtils provides a sample splitting tool that's very handy.
using MLDataUtils

# Set a seed for reproducibility.
using Random
Random.seed!(0);

┌ Info: Recompiling stale cache file /home/cameron/.julia/compiled/v1.0/MLDataUtils/CQWB9.ji for MLDataUtils [cc2ba9b6-d476-5e6d-8eaf-a92d5412d41d]
└ @ Base loading.jl:1184
┌ Warning: Replacing docs for `MLLabelUtils.labelmap :: Union{}` in module `MLLabelUtils`
└ @ Base.Docs docs/Docs.jl:223


In [3]:
# Import the "Default" dataset.
data = RDatasets.dataset("datasets", "mtcars");

# Show the first six rows of the dataset.
head(data)

,Model,MPG,Cyl,Disp,HP,DRat,WT,QSec,VS,AM,Gear,Carb
,String⍰,Float64⍰,Int64⍰,Float64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,Mazda RX4,21.0,6,160.0,110,3.9,2.62,16.46,0,1,4,4
2,Mazda RX4 Wag,21.0,6,160.0,110,3.9,2.875,17.02,0,1,4,4
3,Datsun 710,22.8,4,108.0,93,3.85,2.32,18.61,1,1,4,1
4,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
5,Hornet Sportabout,18.7,8,360.0,175,3.15,3.44,17.02,0,0,3,2
6,Valiant,18.1,6,225.0,105,2.76,3.46,20.22,1,0,3,1


In [132]:
# Split our dataset 5/95 into training/test sets.
train, test = MLDataUtils.splitobs(data, at = 0.7);

# Create our labels. These are the values we are trying to predict.
train_label = train[:MPG]
test_label = test[:MPG]

# Get the list of columns to keep.
remove_names = filter(x->!in(x, [:MPG, :Model]), names(data))

# Filter the test and train sets.
train = Matrix(train[remove_names]);
test = Matrix(test[remove_names]);

# Rescale our matrices.
train = (train .- mean(train, dims=1)) ./ std(train, dims=1);
test = (test .- mean(test, dims=1)) ./ std(test, dims=1);

In [133]:
using LinearAlgebra

# Bayesian logistic regression (LR)
@model linear_regression(x, y, n_obs, n_vars) = begin
    σ₂ ~ InverseGamma(2,3)
    intercept ~ Normal(0, 1)
    coefficients = TArray{Real}(undef, n_vars)
    for i in eachindex(coefficients)
        coefficients[i] ~ Normal(0, 1)
    end

    for i = 1:n_obs
        v = intercept + coefficients ⋅ x[i,:]
        y[i] ~ Normal(v, σ₂)
    end
end;

n_obs, n_vars = size(train)
model = linear_regression(train, train_label, n_obs, n_vars)
chain = sample(model, HMC(4000, 0.001, 20));

┌ Info:  Assume - `σ₂` is a parameter
└ @ Turing /home/cameron/.julia/packages/Turing/YnwiD/src/core/compiler.jl:135
┌ Info:  Assume - `intercept` is a parameter
└ @ Turing /home/cameron/.julia/packages/Turing/YnwiD/src/core/compiler.jl:135
┌ Info:  Assume - `coefficients` is a parameter
└ @ Turing /home/cameron/.julia/packages/Turing/YnwiD/src/core/compiler.jl:165
┌ Info:  Observe - `y` is an observation
└ @ Turing /home/cameron/.julia/packages/Turing/YnwiD/src/core/compiler.jl:153
[HMC] Sampling...  1%  ETA: 0:02:46
  ϵ:         0.001
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[HMC] Sampling...  1%  ETA: 0:02:45
  ϵ:         0.001
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[HMC] Sampling...  2%  ETA: 0:02:44
  ϵ:         0.001
  α:         1.0
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[HMC] Sampling...  2%  ETA: 0:02:55
  ϵ:         0.001
  α:         0.999997481811194
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[HMC] 

[HMC] Finished with
  Running time        = 202.90597742100087;
  Accept rate         = 1.0;
  #lf / sample        = 19.995;
  #evals / sample     = 21.99475;
  pre-cond. diag mat  = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,....





[HMC] Sampling...100% Time: 0:03:23


In [134]:
function prediction(chain, x)
    _, max_lp = findmax(chain[:lp])

    α = chain[:intercept][max_lp]
    β = chain[:coefficients][max_lp]
#     α = mean(chain[:intercept])
#     β = mean(chain[:coefficients])
    
    return α .+ x * β
end

prediction(chain, test)

10-element Array{Float64,1}:
 -0.8534982210381394
 -1.5599273742783681
  0.7522079880238506
  4.184582977729356 
 -1.1352074743695395
  3.3123587990812178
 -0.3882535092316657
 -2.1977537670110374
 -0.5365707663712607
  5.395168361917026 